In [46]:
import pandas as pd

import numpy as np

from MyMultinomialLogit import MyMLE

from scipy.special import expit

import statsmodels.api as sm

In [47]:
#A function that takes parameters and state values and calculates the probability of each choice

def Prob(params, x):
                
    M,B=x.shape 
    beta =np.array(params[:B])
    T = np.array(params[B:])
    x = np.array(x)
    NT=len(T)
    
    pr=np.zeros([M,NT+1])
    
    for i in range(M):
        for j in range(NT+1):
            if j==0:
                pr[i,j]=expit(T[j]-x[i,:].dot(beta))
            if 0<j<NT:
                pr[i,j]=expit(T[j]-x[i,:].dot(beta))-expit(T[j-1]-x[i,:].dot(beta))
            if j==NT:
                pr[i,j]=1-expit(T[j-1]-x[i,:].dot(beta))

    return pr

In [48]:
Data=pd.read_stata('Data.dta')

Players=np.unique(Data['Players'])

Causes=np.unique(Data['Cause'])

Data=Data.set_index(['Players', 'Cause'])

In [49]:
yCols=['PhIII']

yCols_Names=['y']

y_max=2


xCols=['MPhII','zIII']

xCols_Names=['Mu','z']


HfCol='HFIII'

RCol='Revenue2017'

RdCol='RD2017'

In [50]:
y = pd.DataFrame(Data, columns = yCols)

y.columns = yCols_Names

y=np.log(y+1).round()

y[y > y_max] = y_max


x = pd.DataFrame(Data, columns = xCols)

x.columns = xCols_Names

x=np.log(x+1)


Y=np.unique(y)

NT=len(Y)-1

In [51]:
Initial_beta= np.zeros(x.shape[1])
Initial_C=np.linspace(0, (NT-1)/2, num=NT)
start_params =np.append(Initial_beta, Initial_C)
   
extra_params_names = ['C-%s' % i for i in range(NT)]
    
Model = MyMLE(y, x, extra_params_names=extra_params_names)
Result = Model.fit(start_params=start_params,method='newton', maxiter=500)
params=Result.params

print(Result.summary())

Optimization terminated successfully.
         Current function value: 0.395638
         Iterations 10
                                MyMLE Results                                 
Dep. Variable:                      y   Log-Likelihood:                -790.09
Model:                          MyMLE   AIC:                             1584.
Method:            Maximum Likelihood   BIC:                             1595.
Date:                Mon, 08 Jun 2020                                         
Time:                        18:21:08                                         
No. Observations:                1997                                         
Df Residuals:                    1995                                         
Df Model:                           1                                         
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
Mu             0.6586      0

In [52]:
PDF=Prob(params,x)

EY=pd.DataFrame(x, columns = [])

EY['Eyi']=(PDF*Y).sum(axis=1)

EY['Delta']=0

for cs in Causes:
    players=EY.loc[(Players,cs),:].index.get_level_values('Players')
    SEY=EY.loc[(Players,cs),'Eyi'].sum()
    for pl in players:    
        EY.loc[(pl,cs),'Delta']=SEY-EY.loc[(pl,cs),'Eyi']
        
x['Delta']=EY['Delta']

In [53]:
Initial_C=np.linspace(0, (NT-1)/2, num=NT)
start_params =np.append(np.zeros(x.shape[1]), Initial_C)
extra_params_names = ['C-%s' % i for i in range(NT)]
    
Model = MyMLE(y, x, extra_params_names=extra_params_names)
Result = Model.fit(start_params=start_params,method='newton', maxiter=500)
Params=Result.params

print(Result.summary())

Optimization terminated successfully.
         Current function value: 0.391245
         Iterations 10
                                MyMLE Results                                 
Dep. Variable:                      y   Log-Likelihood:                -781.32
Model:                          MyMLE   AIC:                             1569.
Method:            Maximum Likelihood   BIC:                             1585.
Date:                Mon, 08 Jun 2020                                         
Time:                        18:21:17                                         
No. Observations:                1997                                         
Df Residuals:                    1994                                         
Df Model:                           2                                         
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
Mu             1.0036      0

In [18]:
Parameters=pd.DataFrame(np.column_stack((Model.exog_names,Result.params,Result.conf_int())),columns=['variable','Coefficients','[0.025','0.975]'])
Parameters[['Coefficients','[0.025','0.975]']]=Parameters[['Coefficients','[0.025','0.975]']].astype(float).round(2)
Parameters=Parameters.set_index('variable')

Parameters.to_latex('Del.tex')

In [19]:
Hf=Data[HfCol]

R=Data[RCol]

Rd=Data[RdCol]

R=np.log(R)

Rd=np.log(Rd)

In [20]:
x['Delta_Hf']=x['Delta']*Hf

In [21]:
Initial_Th=np.linspace(0, (NT-1)/2, num=NT)
start_params =np.append(np.zeros(x.shape[1]), Initial_Th)
extra_params_names = ['C-%s' % i for i in range(NT)]
    
Model = MyMLE(y, x, extra_params_names=extra_params_names)
Result = Model.fit(start_params=start_params,method='newton', maxiter=500)
Params=Result.params

print(Result.summary())

Optimization terminated successfully.
         Current function value: 0.391245
         Iterations 12
                                MyMLE Results                                 
Dep. Variable:                      y   Log-Likelihood:                -781.32
Model:                          MyMLE   AIC:                             1571.
Method:            Maximum Likelihood   BIC:                             1593.
Date:                Fri, 10 Apr 2020                                         
Time:                        12:54:10                                         
No. Observations:                1997                                         
Df Residuals:                    1993                                         
Df Model:                           3                                         
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
Mu             1.0035      0

In [22]:
Parameters=pd.DataFrame(np.column_stack((Model.exog_names,Result.params,Result.conf_int())),columns=['variable','Coefficients','[0.025','0.975]'])
Parameters[['Coefficients','[0.025','0.975]']]=Parameters[['Coefficients','[0.025','0.975]']].astype(float).round(2)
Parameters=Parameters.set_index('variable')

Parameters.to_latex('Del_Hf.tex')

In [23]:
x=x.drop(columns=['Delta','Delta_Hf'])

x['R']=R

In [24]:
Initial_beta= np.zeros(x.shape[1])
Initial_C=np.linspace(0, (NT-1)/2, num=NT)
start_params =np.append(Initial_beta, Initial_C)
   
extra_params_names = ['C-%s' % i for i in range(NT)]
    
Model = MyMLE(y, x, extra_params_names=extra_params_names)
Result = Model.fit(start_params=start_params,method='newton', maxiter=500)
params=Result.params

print(Result.summary())

Optimization terminated successfully.
         Current function value: 0.395523
         Iterations 10
                                MyMLE Results                                 
Dep. Variable:                      y   Log-Likelihood:                -789.86
Model:                          MyMLE   AIC:                             1586.
Method:            Maximum Likelihood   BIC:                             1603.
Date:                Fri, 10 Apr 2020                                         
Time:                        12:54:17                                         
No. Observations:                1997                                         
Df Residuals:                    1994                                         
Df Model:                           2                                         
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
Mu             0.6565      0

In [25]:
PDF=Prob(params,x)

EY=pd.DataFrame(x, columns = [])

EY['Eyi']=(PDF*Y).sum(axis=1)

EY['Delta']=0

for cs in Causes:
    players=EY.loc[(Players,cs),:].index.get_level_values('Players')
    SEY=EY.loc[(Players,cs),'Eyi'].sum()
    for pl in players:    
        EY.loc[(pl,cs),'Delta']=SEY-EY.loc[(pl,cs),'Eyi']
        
x['Delta']=EY['Delta']

In [26]:
Initial_Th=np.linspace(0, (NT-1)/2, num=NT)
start_params =np.append(np.zeros(x.shape[1]), Initial_Th)
extra_params_names = ['C-%s' % i for i in range(NT)]
    
Model = MyMLE(y, x, extra_params_names=extra_params_names)
Result = Model.fit(start_params=start_params,method='newton', maxiter=500)
Params=Result.params

print(Result.summary())

Optimization terminated successfully.
         Current function value: 0.391202
         Iterations 10
                                MyMLE Results                                 
Dep. Variable:                      y   Log-Likelihood:                -781.23
Model:                          MyMLE   AIC:                             1570.
Method:            Maximum Likelihood   BIC:                             1593.
Date:                Fri, 10 Apr 2020                                         
Time:                        12:54:28                                         
No. Observations:                1997                                         
Df Residuals:                    1993                                         
Df Model:                           3                                         
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
Mu             0.9997      0

In [27]:
Parameters=pd.DataFrame(np.column_stack((Model.exog_names,Result.params,Result.conf_int())),columns=['variable','Coefficients','[0.025','0.975]'])
Parameters[['Coefficients','[0.025','0.975]']]=Parameters[['Coefficients','[0.025','0.975]']].astype(float).round(2)
Parameters=Parameters.set_index('variable')

Parameters.to_latex('Del_R.tex')

In [28]:
FirmSpec=pd.DataFrame(Data, columns = [RCol,HfCol]).reset_index(level='Cause', drop=True).drop_duplicates()

Choice_columns=['Y_'+str(i) for i in range(len(Y))]

Choices = pd.DataFrame(FirmSpec, columns = [])

Choices = Choices.reindex(columns = Choice_columns) 

for pl in Players:
    ypl=y.loc[(pl,Causes),'y']
    ch=np.zeros(len(Y))
    for yy in Y:
        ch[int(yy)]=len(ypl[ypl==yy])
    Choices.loc[pl]=ch
    
    
FirmSpec=pd.concat([FirmSpec,Choices], axis=1)

FirmSpec[HfCol]=FirmSpec[HfCol].astype(float).round(3)

FirmSpec[RCol]=FirmSpec[RCol].astype(float).round(2)

for Ch in Choice_columns:
    FirmSpec[Ch]=FirmSpec[Ch].astype(int)
    
FirmSpec=FirmSpec.rename(columns ={HfCol:'Hf', RCol:'Revenue'})

FirmSpec.to_latex('Players.tex')

FirmSpec

,Revenue,Hf,Y_0,Y_1,Y_2
Players,,,,,
AbbVie,28.22,0.072,39,4,2
Aerie,0.02,1.000,12,1,0
Alkermes,0.90,0.157,16,5,0
Allergan,15.94,0.052,57,21,2
Almirall,0.70,0.164,7,4,1
Amgen,22.78,0.087,10,5,1
Astellas,12.06,0.047,13,3,0
AstraZeneca,22.53,0.045,35,17,1
Bausch Health Companies,8.72,0.080,42,12,1


In [29]:
y['y'].value_counts()

0.0    1531
1.0     416
2.0      50
Name: y, dtype: int64

In [30]:
yy = pd.DataFrame(Data, columns = yCols)

yy.columns = yCols_Names

yy['y'].value_counts().to_latex('Value-count.tex')

yy['y'].value_counts()

0.0     1531
1.0      253
2.0      113
3.0       50
4.0       19
5.0       13
6.0        8
7.0        3
12.0       2
8.0        2
9.0        2
11.0       1
Name: y, dtype: int64

In [31]:
len(yy.y[yy.y>3])

50

In [32]:
y_max=4

y = pd.DataFrame(Data, columns = yCols)

y.columns = yCols_Names

y[y > y_max] = y_max


x = pd.DataFrame(Data, columns = xCols)

x.columns = xCols_Names

Y=np.unique(y)

NT=len(Y)-1

In [33]:
Initial_beta= np.zeros(x.shape[1])
Initial_C=np.linspace(0, (NT-1)/2, num=NT)
start_params =np.append(Initial_beta, Initial_C)
   
extra_params_names = ['C-%s' % i for i in range(NT)]
    
Model = MyMLE(y, x, extra_params_names=extra_params_names)
Result = Model.fit(start_params=start_params,method='newton', maxiter=500)
params=Result.params

print(Result.summary())

Optimization terminated successfully.
         Current function value: 0.648646
         Iterations 7
                                MyMLE Results                                 
Dep. Variable:                      y   Log-Likelihood:                -1295.3
Model:                          MyMLE   AIC:                             2595.
Method:            Maximum Likelihood   BIC:                             2606.
Date:                Fri, 10 Apr 2020                                         
Time:                        12:54:38                                         
No. Observations:                1997                                         
Df Residuals:                    1995                                         
Df Model:                           1                                         
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
Mu             0.0102      0.

In [34]:
PDF=Prob(params,x)

EY=pd.DataFrame(x, columns = [])

EY['Eyi']=(PDF*Y).sum(axis=1)

EY['Delta']=0

for cs in Causes:
    players=EY.loc[(Players,cs),:].index.get_level_values('Players')
    SEY=EY.loc[(Players,cs),'Eyi'].sum()
    for pl in players:    
        EY.loc[(pl,cs),'Delta']=SEY-EY.loc[(pl,cs),'Eyi']
        
x['Delta']=EY['Delta']

In [35]:
Initial_C=np.linspace(0, (NT-1)/2, num=NT)
start_params =np.append(np.zeros(x.shape[1]), Initial_C)
extra_params_names = ['C-%s' % i for i in range(NT)]
    
Model = MyMLE(y, x, extra_params_names=extra_params_names)
Result = Model.fit(start_params=start_params,method='newton', maxiter=500)
Params=Result.params

print(Result.summary())

Optimization terminated successfully.
         Current function value: 0.644512
         Iterations 7
                                MyMLE Results                                 
Dep. Variable:                      y   Log-Likelihood:                -1287.1
Model:                          MyMLE   AIC:                             2580.
Method:            Maximum Likelihood   BIC:                             2597.
Date:                Fri, 10 Apr 2020                                         
Time:                        12:54:49                                         
No. Observations:                1997                                         
Df Residuals:                    1994                                         
Df Model:                           2                                         
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
Mu             0.0215      0.

In [36]:
Parameters=pd.DataFrame(np.column_stack((Model.exog_names,Result.params,Result.conf_int())),columns=['variable','Coefficients','[0.025','0.975]'])
Parameters[['Coefficients','[0.025','0.975]']]=Parameters[['Coefficients','[0.025','0.975]']].astype(float).round(2)
Parameters=Parameters.set_index('variable')

Parameters.to_latex('Del_AltY.tex')

In [37]:
"""
RR = pd.DataFrame(Data, columns = [])

RR['R']=Data[Rd_Col]

RR['Delta_R']=0

for cs in Causes:
    players=RR.loc[(Players,cs),:].index.get_level_values('Players')
    
    for pl in players:
        RR.loc[(pl,cs),'Delta_R']=((RR.loc[(pl,cs),'R']/RR.loc[(Players,cs),'R'])*EY.loc[(Players,cs),'Eyi']).sum()
        
x['Delta_RR']=RR['Delta_R']

"""


"\nRR = pd.DataFrame(Data, columns = [])\n\nRR['R']=Data[Rd_Col]\n\nRR['Delta_R']=0\n\nfor cs in Causes:\n    players=RR.loc[(Players,cs),:].index.get_level_values('Players')\n    \n    for pl in players:\n        RR.loc[(pl,cs),'Delta_R']=((RR.loc[(pl,cs),'R']/RR.loc[(Players,cs),'R'])*EY.loc[(Players,cs),'Eyi']).sum()\n        \nx['Delta_RR']=RR['Delta_R']\n\n"

In [38]:
"""
yy=pd.DataFrame(Data, columns = ['MPhII'])

yy=np.log(yy+1)

xx=pd.DataFrame(Data, columns = ['zIII'])

xx=np.log(xx+1)

model = sm.OLS(yy, xx).fit()

model.summary()

nu=model.resid

yCols=['PhIII']

yCols_Names=['y']

y_max=2


xCols_Names=['Nu']


y = pd.DataFrame(Data, columns = yCols)

y.columns = yCols_Names

y=np.log(y+1).round()

y[y > y_max] = y_max


x = pd.DataFrame(nu)

x.columns = xCols_Names

x['z']=np.log(Data['zIII']+1)

Y=np.unique(y)

NT=len(Y)-1
"""

"\nyy=pd.DataFrame(Data, columns = ['MPhII'])\n\nyy=np.log(yy+1)\n\nxx=pd.DataFrame(Data, columns = ['zIII'])\n\nxx=np.log(xx+1)\n\nmodel = sm.OLS(yy, xx).fit()\n\nmodel.summary()\n\nnu=model.resid\n\nyCols=['PhIII']\n\nyCols_Names=['y']\n\ny_max=2\n\n\nxCols_Names=['Nu']\n\n\ny = pd.DataFrame(Data, columns = yCols)\n\ny.columns = yCols_Names\n\ny=np.log(y+1).round()\n\ny[y > y_max] = y_max\n\n\nx = pd.DataFrame(nu)\n\nx.columns = xCols_Names\n\nx['z']=np.log(Data['zIII']+1)\n\nY=np.unique(y)\n\nNT=len(Y)-1\n"